# OWL-Server MCP Client Example

This notebook demonstrates how to use the FastMCP client to interact with the OWL-Server. The Model Context Protocol (MCP) provides a standardized way for applications to expose their functionality to AI assistants and other clients.

In this example, we'll show how to:
1. Connect to an OWL-Server running as an MCP server
2. Create and manipulate OWL ontologies
3. Query ontology information
4. Work with prefix mappings
5. Use the configuration system to manage ontologies

## Setup and Installation

First, make sure you have the required packages installed:

In [ ]:
# Install required packages
!pip install fastmcp py-horned-owl nest-asyncio

We need to use `nest_asyncio` to allow running async code in Jupyter notebooks:

In [ ]:
import nest_asyncio
import asyncio
import tempfile
import os
from fastmcp import Client
from pprint import pprint

# Enable nested asyncio for Jupyter notebooks
nest_asyncio.apply()

## Starting the OWL-Server

Before connecting with a client, you need to have the OWL-Server running. In a separate terminal, you would run:

```bash
python -m owl_mcp.mcp_tools
```

For this notebook example, we'll use a subprocess to start the server:

In [ ]:
import subprocess
import time
import signal

# Start the MCP server in a subprocess
print("Starting OWL-Server MCP server...")
server_process = subprocess.Popen(
    ["python", "-m", "owl_mcp.mcp_tools"],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
    bufsize=1
)

# Give the server a moment to start up
time.sleep(2)
print("Server started!")

## Creating a Temporary OWL File

For this demonstration, we'll create a temporary OWL file to work with:

In [ ]:
# Create a temporary OWL file
temp_dir = tempfile.gettempdir()
owl_file_path = os.path.join(temp_dir, "example_ontology.owl")

# Create an empty file (the OWL-Server will initialize it properly)
with open(owl_file_path, "w") as f:
    f.write("Ontology()")

print(f"Created temporary OWL file at: {owl_file_path}")

## Connecting to the OWL-Server with FastMCP Client

Now we can connect to the OWL-Server using the FastMCP client:

In [ ]:
async def connect_to_server():
    # Connect to the server subprocess
    # Note: In a real application, you would connect to the server's endpoint
    # For example: client = Client("https://example.com/mcp") or Client("ws://localhost:9000")
    # For this example, we're using the subprocess we started above
    client = Client(server_process)
    
    async with client:
        # List available tools
        tools = await client.list_tools()
        print("Available tools:")
        for tool in tools:
            print(f"- {tool.name}: {tool.description}")
            
        return client

# Run the async function
client = asyncio.run(connect_to_server())

## Working with the Ontology

Now we can start working with our ontology through the MCP interface. First, let's add some prefix mappings:

In [ ]:
async def add_prefixes():
    async with Client(server_process) as client:
        # Add prefix mappings
        prefixes = [
            ("ex", "http://example.org/"),
            ("pizza", "http://www.co-ode.org/ontologies/pizza/pizza.owl#"),
            ("ont", "http://www.my-ontology.com/"),
        ]
        
        print("Adding prefix mappings...")
        for prefix, uri in prefixes:
            result = await client.call_tool("add_prefix", {
                "owl_file_path": owl_file_path,
                "prefix": prefix,
                "uri": uri
            })
            print(f"  {result.text}")

# Run the async function
asyncio.run(add_prefixes())

Now let's add some axioms to our ontology:

In [ ]:
async def add_pizza_axioms():
    async with Client(server_process) as client:
        print("Building a Pizza ontology...")
        
        # Define classes
        classes = [
            "Declaration(Class(pizza:Pizza))",
            "Declaration(Class(pizza:PizzaBase))",
            "Declaration(Class(pizza:PizzaTopping))",
            "Declaration(Class(pizza:CheeseTopping))",
            "Declaration(Class(pizza:MeatTopping))",
            "Declaration(Class(pizza:VegetableTopping))",
            "Declaration(Class(pizza:Margherita))",
            "Declaration(Class(pizza:Pepperoni))",
            "Declaration(Class(pizza:Vegetarian))"
        ]
        
        # Add class declarations
        for class_axiom in classes:
            result = await client.call_tool("add_axiom", {
                "owl_file_path": owl_file_path,
                "axiom_str": class_axiom
            })
            print(f"  {result.text}")
        
        # Define subclass relationships
        subclass_axioms = [
            "SubClassOf(pizza:CheeseTopping pizza:PizzaTopping)",
            "SubClassOf(pizza:MeatTopping pizza:PizzaTopping)",
            "SubClassOf(pizza:VegetableTopping pizza:PizzaTopping)",
            "SubClassOf(pizza:Margherita pizza:Pizza)",
            "SubClassOf(pizza:Pepperoni pizza:Pizza)",
            "SubClassOf(pizza:Vegetarian pizza:Pizza)"
        ]
        
        # Add subclass axioms
        for subclass_axiom in subclass_axioms:
            result = await client.call_tool("add_axiom", {
                "owl_file_path": owl_file_path,
                "axiom_str": subclass_axiom
            })
            print(f"  {result.text}")
        
        # Define object properties
        object_properties = [
            "Declaration(ObjectProperty(pizza:hasTopping))",
            "Declaration(ObjectProperty(pizza:hasBase))"
        ]
        
        # Add object properties
        for property_axiom in object_properties:
            result = await client.call_tool("add_axiom", {
                "owl_file_path": owl_file_path,
                "axiom_str": property_axiom
            })
            print(f"  {result.text}")
        
        # Define property domains and ranges
        property_restrictions = [
            "ObjectPropertyDomain(pizza:hasTopping pizza:Pizza)",
            "ObjectPropertyRange(pizza:hasTopping pizza:PizzaTopping)",
            "ObjectPropertyDomain(pizza:hasBase pizza:Pizza)",
            "ObjectPropertyRange(pizza:hasBase pizza:PizzaBase)"
        ]
        
        # Add property restrictions
        for restriction in property_restrictions:
            result = await client.call_tool("add_axiom", {
                "owl_file_path": owl_file_path,
                "axiom_str": restriction
            })
            print(f"  {result.text}")

# Run the async function
asyncio.run(add_pizza_axioms())

## Querying the Ontology

Now that we have built our ontology, let's query it to find specific axioms:

In [ ]:
async def query_ontology():
    async with Client(server_process) as client:
        print("Querying for pizza classes...")
        
        # Find all axioms containing 'Pizza'
        result = await client.call_tool("find_axioms", {
            "owl_file_path": owl_file_path,
            "pattern": "Pizza"
        })
        
        # Output is a list of strings
        pizza_axioms = result.value
        print(f"Found {len(pizza_axioms)} axioms containing 'Pizza':")
        for i, axiom in enumerate(pizza_axioms, 1):
            print(f"  {i}. {axiom}")
        
        print("\nQuerying for subclass relationships...")
        # Find all SubClassOf axioms
        result = await client.call_tool("find_axioms", {
            "owl_file_path": owl_file_path,
            "pattern": "",
            "limit": 100
        })
        
        # Filter for SubClassOf axioms
        subclass_axioms = [a for a in result.value if a.startswith("SubClassOf")]
        print(f"Found {len(subclass_axioms)} subclass relationships:")
        for i, axiom in enumerate(subclass_axioms, 1):
            print(f"  {i}. {axiom}")
        
        print("\nQuerying for object properties...")
        # Find all ObjectProperty declarations
        result = await client.call_tool("find_axioms", {
            "owl_file_path": owl_file_path,
            "pattern": "ObjectProperty"
        })
        
        obj_props = result.value
        print(f"Found {len(obj_props)} object properties:")
        for i, axiom in enumerate(obj_props, 1):
            print(f"  {i}. {axiom}")

# Run the async function
asyncio.run(query_ontology())

## Modifying the Ontology

Now let's modify our ontology by adding and removing some axioms:

In [ ]:
async def modify_ontology():
    async with Client(server_process) as client:
        print("Adding a new pizza type...")
        
        # Add a new pizza class
        new_pizza_axioms = [
            "Declaration(Class(pizza:HawaiianPizza))",
            "SubClassOf(pizza:HawaiianPizza pizza:Pizza)",
            "Declaration(Class(pizza:PineappleTopping))",
            "SubClassOf(pizza:PineappleTopping pizza:VegetableTopping)",
            "Declaration(Class(pizza:HamTopping))",
            "SubClassOf(pizza:HamTopping pizza:MeatTopping)"
        ]
        
        for axiom in new_pizza_axioms:
            result = await client.call_tool("add_axiom", {
                "owl_file_path": owl_file_path,
                "axiom_str": axiom
            })
            print(f"  {result.text}")
        
        # Let's see if our new pizza is there
        print("\nVerifying new pizza was added...")
        result = await client.call_tool("find_axioms", {
            "owl_file_path": owl_file_path,
            "pattern": "Hawaiian"
        })
        
        hawaiian_axioms = result.value
        print(f"Found {len(hawaiian_axioms)} axioms about Hawaiian pizza:")
        for axiom in hawaiian_axioms:
            print(f"  {axiom}")
        
        # Now remove the Hawaiian pizza (controversial!)
        print("\nRemoving the controversial Hawaiian pizza...")
        for axiom in hawaiian_axioms:
            result = await client.call_tool("remove_axiom", {
                "owl_file_path": owl_file_path,
                "axiom_str": axiom
            })
            print(f"  {result.text}")
        
        # Verify it was removed
        print("\nVerifying Hawaiian pizza was removed...")
        result = await client.call_tool("find_axioms", {
            "owl_file_path": owl_file_path,
            "pattern": "Hawaiian"
        })
        
        remaining_axioms = result.value
        if remaining_axioms:
            print(f"Found {len(remaining_axioms)} remaining axioms about Hawaiian pizza:")
            for axiom in remaining_axioms:
                print(f"  {axiom}")
        else:
            print("Hawaiian pizza successfully removed from the ontology.")

# Run the async function
asyncio.run(modify_ontology())

## Working with Ontology Metadata

Let's add and query some ontology metadata:

In [ ]:
async def work_with_metadata():
    async with Client(server_process) as client:
        print("Adding ontology metadata...")
        
        # Add ontology annotations
        metadata_axioms = [
            "AnnotationAssertion(rdfs:label ont:PizzaOntology \"Pizza Ontology\")",
            "AnnotationAssertion(rdfs:comment ont:PizzaOntology \"An example ontology about pizzas created via MCP client\")",
            "AnnotationAssertion(owl:versionInfo ont:PizzaOntology \"1.0.0\")"
        ]
        
        for axiom in metadata_axioms:
            result = await client.call_tool("add_axiom", {
                "owl_file_path": owl_file_path,
                "axiom_str": axiom
            })
            print(f"  {result.text}")
        
        # Retrieve metadata
        print("\nRetrieving ontology metadata...")
        result = await client.call_tool("ontology_metadata", {
            "owl_file_path": owl_file_path
        })
        
        metadata = result.value
        print(f"Found {len(metadata)} metadata items:")
        for item in metadata:
            print(f"  {item}")

# Run the async function
asyncio.run(work_with_metadata())

## Listing Active OWL Files

Let's see which OWL files are currently being managed by the server:

In [ ]:
async def list_active_files():
    async with Client(server_process) as client:
        # Access a resource
        print("Listing active OWL files...")
        result = await client.read_resource("resource://active")
        
        active_files = result.value
        if active_files:
            print(f"The server is managing {len(active_files)} OWL files:")
            for file in active_files:
                print(f"  {file}")
        else:
            print("No active OWL files found.")

# Run the async function
asyncio.run(list_active_files())

## Working with the Configuration System

Now let's explore the configuration system that allows you to manage named ontologies:

In [ ]:
async def work_with_config():
    async with Client(server_process) as client:
        # Create a new temporary file for our pizza ontology
        pizza_file = os.path.join(temp_dir, "pizza_ontology.owl")
        with open(pizza_file, "w") as f:
            f.write("Ontology()")
        
        try:
            print("Registering the pizza ontology in the configuration...")
            # Register this ontology in the configuration with a name
            result = await client.call_tool("load_and_register_ontology", {
                "owl_file_path": pizza_file,
                "name": "pizza",
                "description": "Pizza ontology example",
                "readonly": False,
                "metadata_axioms": [
                    "AnnotationAssertion(rdfs:label ont:PizzaOntology \"Pizza Ontology\")",
                    "AnnotationAssertion(owl:versionInfo ont:PizzaOntology \"1.0.0\")"
                ]
            })
            print(f"  {result.text}")
            
            # Add some basic axioms to the pizza ontology
            axioms = [
                "Declaration(Class(pizza:Pizza))",
                "Declaration(Class(pizza:Topping))",
                "SubClassOf(pizza:Pizza owl:Thing)"
            ]
            
            print("\nAdding axioms to the pizza ontology using its name...")
            for axiom in axioms:
                result = await client.call_tool("add_axiom_by_name", {
                    "ontology_name": "pizza",
                    "axiom_str": axiom
                })
                print(f"  {result.text}")
                
            # Let's see what ontologies we have in our configuration
            print("\nFetching all configured ontologies...")
            result = await client.read_resource("resource://config/ontologies")
            
            ontologies = result.value
            print(f"Found {len(ontologies)} configured ontologies:")
            for ontology in ontologies:
                print(f"  - {ontology['name']}: {ontology['path']}")
                if ontology['description']:
                    print(f"    Description: {ontology['description']}")
                print(f"    Read-only: {ontology['readonly']}")
                
            # Get details about the pizza ontology
            print("\nGetting details about the pizza ontology...")
            result = await client.read_resource("resource://config/ontology/pizza")
            
            pizza_config = result.value
            if pizza_config:
                print(f"  Path: {pizza_config['path']}")
                print(f"  Read-only: {pizza_config['readonly']}")
                print(f"  Metadata axioms: {len(pizza_config['metadata_axioms'])}")
                
            # Query axioms using the named ontology
            print("\nQuerying axioms in the pizza ontology by name...")
            result = await client.call_tool("find_axioms_by_name", {
                "ontology_name": "pizza",
                "pattern": "Pizza"
            })
            
            pizza_axioms = result.value
            print(f"Found {len(pizza_axioms)} axioms about Pizza:")
            for axiom in pizza_axioms:
                print(f"  {axiom}")
                
            # Cleanup
            print("\nRemoving the pizza ontology from configuration...")
            result = await client.call_tool("remove_ontology_config", {
                "name": "pizza"
            })
            print(f"  {result.text}")
            
        finally:
            # Make sure we clean up the temp file
            if os.path.exists(pizza_file):
                os.remove(pizza_file)
                print(f"Removed temporary file: {pizza_file}")

# Run the async function
asyncio.run(work_with_config())

## Cleaning Up

Finally, let's stop the OWL service for our file and clean up resources:

In [ ]:
async def cleanup():
    try:
        async with Client(server_process) as client:
            # Stop the OWL service for our file
            print("Stopping OWL service for our file...")
            result = await client.call_tool("stop_owl_service", {
                "owl_file_path": owl_file_path
            })
            print(f"  {result.text}")
            
            # Verify it was stopped
            print("\nVerifying file is no longer active...")
            result = await client.read_resource("resource://active")
            active_files = result.value
            
            if owl_file_path in active_files:
                print(f"Warning: {owl_file_path} is still active!")
            else:
                print(f"{owl_file_path} is no longer active.")
            
    finally:
        # Shut down the server process
        print("\nShutting down the server process...")
        server_process.terminate()
        server_process.wait(timeout=5)
        print("Server process terminated.")
        
        # Remove the temporary OWL file
        if os.path.exists(owl_file_path):
            os.remove(owl_file_path)
            print(f"Removed temporary file: {owl_file_path}")

# Run the async function
asyncio.run(cleanup())

## Conclusion

In this notebook, we demonstrated how to:

1. Connect to an OWL-Server MCP server using the FastMCP client
2. Create and manipulate an OWL ontology programmatically
3. Add and remove axioms in OWL functional syntax
4. Query for specific axioms in the ontology
5. Work with ontology metadata
6. Use the configuration system to register and manage named ontologies
7. Access configuration through MCP resources
8. Manage active OWL files and clean up resources

The Model Context Protocol provides a clean, standardized way for clients to interact with the OWL-Server, enabling seamless integration with AI assistants and other applications. This approach abstracts away the complexity of low-level ontology manipulation, allowing developers to focus on their domain-specific tasks.